In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# تابع برای محاسبه SSIM
def calculate_ssim(image1, image2):
    if len(image1.shape) == 3:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    return ssim(image1, image2)

# تابع برای مقایسه نقاط کلیدی با استفاده از ORB
def calculate_orb_similarity(image1, image2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(image1, None)
    kp2, des2 = orb.detectAndCompute(image2, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    if des1 is None or des2 is None:
        return 0  # در صورتی که نقاط کلیدی یافت نشود
    matches = bf.match(des1, des2)
    return len(matches) / max(len(kp1), len(kp2))

def calculate_frequency_similarity(image1, image2):
    if len(image1.shape) == 3:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    f1 = np.fft.fft2(image1)
    f2 = np.fft.fft2(image2)
    f1_shift = np.fft.fftshift(f1)
    f2_shift = np.fft.fftshift(f2)
    magnitude1 = np.abs(f1_shift)
    magnitude2 = np.abs(f2_shift)
    # اضافه کردن data_range به تابع SSIM
    return ssim(magnitude1, magnitude2, data_range=magnitude1.max() - magnitude1.min())


# تابع برای مقایسه هیستوگرام رنگ‌ها در فضای HSV
def calculate_color_similarity(image1, image2):
    hsv1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    hist1 = cv2.calcHist([hsv1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([hsv2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

# تابع برای ترکیب معیارهای شباهت
def calculate_combined_similarity(image1, image2):
    ssim_score = calculate_ssim(image1, image2)
    orb_score = calculate_orb_similarity(image1, image2)
    frequency_score = calculate_frequency_similarity(image1, image2)
    color_score = calculate_color_similarity(image1, image2)

    # وزن‌دهی معیارها
    combined_score = (
        0.4 * ssim_score +
        0.5 * orb_score +
        # 0.3 * frequency_score +
        0 * color_score
    )
    return combined_score

# تابع چرخش تصویر
def rotate_image(img, angle):
    if angle == 90:
        return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(img, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return img

# تقسیم تصویر به 16 قسمت
def split_image(img):
    h, w = img.shape[:2]
    h_step, w_step = h // 4, w // 4
    patches = []
    coords = []
    for i in range(4):
        for j in range(4):
            patch = img[i * h_step:(i + 1) * h_step, j * w_step:(j + 1) * w_step]
            patches.append(patch)
            coords.append((i * h_step, (i + 1) * h_step, j * w_step, (j + 1) * w_step))
    return patches, coords

# خواندن تصاویر
image1 = cv2.imread(r"C:\Users\hassa\OneDrive\Desktop\test\17.jpg" )
image2 = cv2.imread(r"C:\Users\hassa\OneDrive\Desktop\test\18.jpg")

# تغییر اندازه تصویر اول به ابعاد تصویر دوم
h2, w2 = image2.shape[:2]
image1 = cv2.resize(image1, (w2, h2))

# تقسیم تصاویر به 16 قسمت
patches1, coords1 = split_image(image1)
patches2, coords2 = split_image(image2)

# نگاشت مختصات به شماره
coords_to_index = {coord: idx + 1 for idx, coord in enumerate(coords2)}

# پیدا کردن بهترین تطبیق‌ها
selected_patches = []
mapping_info = []
used_patches = set()

for i, patch1 in enumerate(patches1):
    max_similarity = -1
    best_patch_coords = None
    best_rotation = 0
    best_patch_idx = -1
    for j, patch2 in enumerate(patches2):
        if j in used_patches:
            continue
        for angle in [0, 90, 180, 270]:
            rotated_patch = rotate_image(patch2, angle)
            similarity = calculate_combined_similarity(patch1, rotated_patch)
            if similarity > max_similarity:
                max_similarity = similarity
                best_patch_coords = coords2[j]
                best_rotation = angle
                best_patch_idx = j
    if best_patch_coords is not None:
        selected_patches.append((best_patch_coords, best_rotation))
        mapping_info.append((coords_to_index[coords1[i]], coords_to_index[best_patch_coords], best_rotation))
        used_patches.add(best_patch_idx)

# نمایش اطلاعات مختصات و چرخش
print("Mapping Information:")
for idx, (source_idx, dest_idx, rotation) in enumerate(mapping_info):
    print(f"Patch {idx+1}: Source {source_idx} -> Destination {dest_idx}, Rotation: {rotation} degrees")
# ایجاد یک بوم خالی برای چیدمان نهایی
final_image = np.zeros_like(image1)

# قرار دادن قطعات در موقعیت‌های صحیح
for idx, (source_idx, dest_idx, rotation) in enumerate(mapping_info):
    # مختصات تکه انتخاب‌شده
    dest_coords = coords2[dest_idx - 1]
    dest_patch = patches2[dest_idx - 1]
    
    # چرخش تکه انتخاب‌شده
    dest_patch_rotated = rotate_image(dest_patch, rotation)
    
    # قرار دادن تکه در تصویر نهایی
    y1, y2, x1, x2 = coords1[source_idx - 1]
    final_image[y1:y2, x1:x2] = dest_patch_rotated

# نمایش تصویر نهایی
cv2.imshow("Final Image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
